# Rekognition

First we must import the boto3 library and then we can create a Rekognition client.

In [ ]:
import boto3
rekognition = boto3.client("rekognition", "us-east-1")  

## Image Detection

In [ ]:

response = rekognition.detect_labels(
		Image={
			"S3Object": {
				"Bucket": "facesusconygrecom",
				"Name": "sarah.jpg",
			}
		},
		MaxLabels=10,
		MinConfidence=70,
	)

for label in response['Labels']:
    print ("{Name} - {Confidence}%".format(**label))
    


Clothing - 99.8679428100586%
Fashion - 99.45709991455078%
Gown - 99.45709991455078%
Robe - 99.45709991455078%
Evening Dress - 99.45709991455078%
Person - 98.88396453857422%
Railing - 92.95266723632812%
Pants - 83.74752807617188%
Plant - 81.14045715332031%
Alloy Wheel - 77.19818878173828%


## Facial Recognition

In [ ]:
response = rekognition.compare_faces(
	    SourceImage={
			"S3Object": {
				"Bucket": "facesusconygrecom",
				"Name": "sarah.jpg",
			}
		},
		TargetImage={
			"S3Object": {
				"Bucket": "facesusconygrecom",
				"Name": "family.jpg",
			}
		},
	    SimilarityThreshold=90,
	)
for match in response["FaceMatches"]:
	print ("Target Face ({Confidence}%)".format(**match['Face']))

    

Target Face (99.99909210205078%)


# Text Services
## Amazon Translate

In [ ]:
translate = boto3.client(service_name='translate', region_name='us-east-1', use_ssl=True)

result = translate.translate_text(Text="Hello, World", 
            SourceLanguageCode="en", TargetLanguageCode="de")
print('TranslatedText: ' + result.get('TranslatedText'))
print('SourceLanguageCode: ' + result.get('SourceLanguageCode'))
print('TargetLanguageCode: ' + result.get('TargetLanguageCode'))

TranslatedText: Hallo, Welt
SourceLanguageCode: en
TargetLanguageCode: de


## Amazon Polly

In [ ]:
"""Getting Started Example for Python 2.7+/3.3+"""
from boto3 import Session
from botocore.exceptions import BotoCoreError, ClientError
from contextlib import closing
import os
import sys
import subprocess
from tempfile import gettempdir

# Create a client using the credentials and region defined in the [adminuser]
# section of the AWS credentials file (~/.aws/credentials).
session = Session()
polly = session.client("polly")

try:
    # Request speech synthesis
    response = polly.synthesize_speech(Text="Hello world!", OutputFormat="mp3",
                                        VoiceId="Joanna")
except (BotoCoreError, ClientError) as error:
    # The service returned an error, exit gracefully
    print(error)
    sys.exit(-1)

# Access the audio stream from the response
if "AudioStream" in response:
    # Note: Closing the stream is important because the service throttles on the
    # number of parallel connections. Here we are using contextlib.closing to
    # ensure the close method of the stream object will be called automatically
    # at the end of the with statement's scope.
        with closing(response["AudioStream"]) as stream:
            output = os.path.join(gettempdir(), "speech.mp3")

            try:
                # Open a file for writing the output as a binary stream
                with open(output, "wb") as file:
                    file.write(stream.read())
            except IOError as error:
                # Could not write to file, exit gracefully
                print(error)


# Play the audio using the platform's default player
if sys.platform == "win32":
    os.startfile(output)
else:
    # The following works on macOS and Linux. (Darwin = mac, xdg-open = linux).
    opener = "open" if sys.platform == "darwin" else "xdg-open"
    subprocess.call([opener, output])